## SQL

#### Libraries

In [1]:
import sqlite3

#### SQL syntax

In [2]:
# retrieving info
'''SELECT column_name_1, column_name_2 FROM table_name'''

# filtering
# =: eq
# <>: not equal
# >: gt
# <: lt
# >=: gteq
# <=: lteq
# BETWEEN ... AND ...: range
'''SELECT column_name_1, column_name_2 FROM table_name WHERE price > 10'''

# NULL is empty
'''SELECT column_name_1, column_name_2 FROM table_name WHERE price = NULL'''

# order by ASC or DESC
'''SELECT column_name_1, column_name_2 FROM table_name ORDER BY price ASC'''

# distinct values (also possible to have distinct 2 or more columns)
'''SELECT DISTINCT column_name_1 FROM table_name'''

# regex filtering
'''SELECT column_name_1 FROM table_name WHERE major IN ("Physics", "Mathematics")'''
'''SELECT column_name_1 FROM table_name WHERE major NOT IN ("Computer Science")'''
# like ('_' as only 1 char or '%' as any number of char)
# major starts with C
'''SELECT column_name_1 FROM table_name WHERE major LIKE "C%"'''
# major has C_mputer structure, and any char can be instead of _ (not only 1)
'''SELECT column_name_1 FROM table_name WHERE major LIKE "C_mputer"''' 

# aggregate functions (SUM, MIN, MAX, AVG, COUNT)
'''SELECT SUM(column_name_1) FROM table_name''' 

# group by some column
'''SELECT SUM(column_name_1) FROM table_name GROUP BY customer_id''' 
# group by year and count how many entries for each year
'''SELECT Founded, COUNT(*) FROM t_big10west GROUP BY Founded'''

# filtering after group by
# WHERE applied for original table; HAVING applied for a new table and creating another table
'''SELECT SUM(column_name_1) FROM table_name GROUP BY customer_id HAVING price > 10''' 

# AS to save a temporary variable 
'''SELECT SUM(column_name_1) AS sum_of_smt FROM table_name GROUP BY customer_id HAVING price > 10'''

# join several tables:
# (INNER) JOIN: output table will have entries which are only in both tables at the same time
# LEFT JOIN: output table will have entries which are in left tables
# RIGHT JOIN: output table will have entries which are in right tables
# FULL JOIN: output table will have entries which are in left and right tables
'''SELECT table_1_column_3, table_2_column_7 FROM table_name_1 LEFT JOIN table_name_2 ON table_1_column_4 > 10'''

# creating a table. datatype is optional
'''CREATE TABLE table_name [col1 datatype, col2 datatype]'''
# delete table
'''DROP TABLE table_name'''

# insert new row. [col1, col2] is optional, if giving all values
'''INSERT INTO table_name [col1, col2] VALUES value1, value2'''
# update row. WHERE is mandatory!!! (otherwise all rows will be modified)
'''UPDATE table_name SET col1=value1, col3=value3 WHERE user_i = 10'''
# delete row. WHERE is mandatory!!! (otherwise all rows will be deleted)
'''DELETE FROM table_name WHERE user_i = 10'''

'DELETE FROM table_name WHERE user_i = 10'

#### Create SQL table

In [3]:
conn = sqlite3.connect('./help/big10.db')
c = conn.cursor()

# Double quotes are the norm in most of the python programs. 
# Triple quotes are used mostly when you have a big multi-line string 
# copy-pasted from somewhere else and you want to embed them in your program.
c.execute('''CREATE TABLE t_big10west (ID, University, City, State, Founded)''')

universities = [(101, 'University of Illinois', 'Urbana', 'Illinois', 1867),
                (202, 'University of Iowa', 'Iowa City', 'Iowa', 1847),
                (303, 'University of Minnesota', 'Minneapolis', 'Minnesota', 1851),
                (404, 'University of Nebraska', 'Lincoln', 'Nebraska', 1869),
                (505, 'Northwestern University', 'Evanston', 'Illinois', 1851),
                (606, 'Purdue University', 'West Lafayette', 'Indiana', 1869),
                (707, 'University of Wisconsin', 'Madison', 'Wisconsin', 1849)]
c.executemany('INSERT INTO t_big10west VALUES (?,?,?,?,?)', universities)
conn.commit()

# get all rows
for row in c.execute('''SELECT * FROM t_big10west'''):
    print(row)

# print column names
print([desc[0] for desc in c.description])
    
conn.close()

(101, 'University of Illinois', 'Urbana', 'Illinois', 1867)
(202, 'University of Iowa', 'Iowa City', 'Iowa', 1847)
(303, 'University of Minnesota', 'Minneapolis', 'Minnesota', 1851)
(404, 'University of Nebraska', 'Lincoln', 'Nebraska', 1869)
(505, 'Northwestern University', 'Evanston', 'Illinois', 1851)
(606, 'Purdue University', 'West Lafayette', 'Indiana', 1869)
(707, 'University of Wisconsin', 'Madison', 'Wisconsin', 1849)
['ID', 'University', 'City', 'State', 'Founded']


#### Chaning the mistake

In [4]:
conn = sqlite3.connect('./help/big10.db')
c = conn.cursor()

c.execute('UPDATE t_big10west SET Founded=1848 WHERE ID=707')
conn.commit()

conn.close()

#### Get all rows 

In [5]:
conn = sqlite3.connect('./help/big10.db')
c = conn.cursor()

# get all rows
for row in c.execute('SELECT * FROM t_big10west'):
    print(row)
    
conn.close()

(101, 'University of Illinois', 'Urbana', 'Illinois', 1867)
(202, 'University of Iowa', 'Iowa City', 'Iowa', 1847)
(303, 'University of Minnesota', 'Minneapolis', 'Minnesota', 1851)
(404, 'University of Nebraska', 'Lincoln', 'Nebraska', 1869)
(505, 'Northwestern University', 'Evanston', 'Illinois', 1851)
(606, 'Purdue University', 'West Lafayette', 'Indiana', 1869)
(707, 'University of Wisconsin', 'Madison', 'Wisconsin', 1848)
